In [ ]:
!pip install -U ultralytics tqdm opencv-python torchvision pandas Pillow gdown aicrowd-cli

In [ ]:
!pip install tensorflow numpy torch sklearn streamlit joblib

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import os
import cv2
import torch
from pathlib import Path
from timeit import default_timer as timer
from tqdm.auto import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import streamlit as st
import joblib

In [ ]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/ZnzzHTVAlOLDyeGsOOuI-0x6OTFooUBtTZ3SJ6LIxaI
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/ZnzzHTVAlOLDyeGsOOuI-0x6OTFooUBtTZ3SJ6LIxaI'
Login Error: Couldn't login. Max retries exceeded
Please try logging in again


In [ ]:
!aicrowd dataset download --challenge mosquitoalert-challenge-2023

phase2_train_v0.csv: 100% 553k/553k [00:00<00:00, 3.80MB/s]
phase2_train_v0.zip: 100% 10.3G/10.3G [11:57<00:00, 14.4MB/s]


In [ ]:
!unzip -qq phase2_train_v0.zip -d test_images/

In [ ]:
!mv phase2_train_v0.csv test_images/images_train.csv

In [ ]:
csv_file = 'test_images/images_train.csv'
df = pd.read_csv(csv_file)

# Shuffle the dataframe
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

def load_and_preprocess_images(image_paths, target_size):
    images = []
    x = 0
    for path in image_paths:
        x+=1
        image = cv2.imread("test_images/final/"+path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure images are in RGB format
        image = cv2.resize(image, target_size)
        image = image / 255.0  # Normalize pixel values to [0, 1]
        images.append(image)

    return np.array(images)

# Split the data into train, validation, and test sets
train_size = 0.7
validation_size = 0.15
test_size = 0.15

X = load_and_preprocess_images(df['img_fName'], target_size=(64, 64))
y = LabelBinarizer().fit_transform(df['class_label'])

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=1 - train_size, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=test_size / (test_size + validation_size), random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow(X_validation, y_validation, batch_size=batch_size)

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Create a sequential model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output for the fully connected layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(6, activation='softmax'))  # Replace 'number_of_classes' with the actual number of classes in your dataset

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(train_generator, epochs=75, validation_data=validation_generator)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)

print(f'Test accuracy: {test_accuracy * 100:.2f}%')


Epoch 1/75
227/227 [==============================] - 12s 52ms/step - loss: 1.1090 - accuracy: 0.4632 - val_loss: 1.0762 - val_accuracy: 0.4891
Epoch 2/75
227/227 [==============================] - 10s 42ms/step - loss: 1.1043 - accuracy: 0.4550 - val_loss: 1.0856 - val_accuracy: 0.4556
Epoch 3/75
227/227 [==============================] - 12s 51ms/step - loss: 1.1025 - accuracy: 0.4596 - val_loss: 1.0886 - val_accuracy: 0.4434
Epoch 4/75
227/227 [==============================] - 12s 51ms/step - loss: 1.1005 - accuracy: 0.4504 - val_loss: 1.0800 - val_accuracy: 0.4595
Epoch 5/75
227/227 [==============================] - 10s 46ms/step - loss: 1.0934 - accuracy: 0.4688 - val_loss: 1.0782 - val_accuracy: 0.4865
Epoch 6/75
227/227 [==============================] - 12s 53ms/step - loss: 1.0914 - accuracy: 0.4879 - val_loss: 1.0553 - val_accuracy: 0.5624
Epoch 7/75
227/227 [==============================] - 12s 51ms/step - loss: 1.0729 - accuracy: 0.5223 - val_loss: 1.0558 - val_accuracy:

In [ ]:
# Se guarda el modelo
joblib.dump(model, "mosquitos_v1.sav")

['mosquitos_v1.sav']

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow(X_validation, y_validation, batch_size=batch_size)

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
# Create a sequential model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output for the fully connected layers
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(6, activation='softmax'))  # Replace 'number_of_classes' with the actual number of classes in your dataset

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator, epochs=20, validation_data=validation_generator)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)

print(f'Test accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/20
227/227 [==============================] - 14s 53ms/step - loss: 1.4499 - accuracy: 0.4892 - val_loss: 1.6329 - val_accuracy: 0.4344
Epoch 2/20
227/227 [==============================] - 12s 53ms/step - loss: 1.1932 - accuracy: 0.4980 - val_loss: 1.0947 - val_accuracy: 0.4447
Epoch 3/20
227/227 [==============================] - 10s 42ms/step - loss: 1.1681 - accuracy: 0.5078 - val_loss: 1.0663 - val_accuracy: 0.5264
Epoch 4/20
227/227 [==============================] - 12s 54ms/step - loss: 1.1474 - accuracy: 0.5202 - val_loss: 1.1079 - val_accuracy: 0.4717
Epoch 5/20
227/227 [==============================] - 10s 42ms/step - loss: 1.1027 - accuracy: 0.5408 - val_loss: 1.0698 - val_accuracy: 0.5785
Epoch 6/20
227/227 [==============================] - 11s 49ms/step - loss: 1.0777 - accuracy: 0.5699 - val_loss: 0.9859 - val_accuracy: 0.6287
Epoch 7/20
227/227 [==============================] - 13s 57ms/step - loss: 1.0370 - accuracy: 0.6013 - val_loss: 1.2755 - val_accuracy:

In [ ]:
# Se guarda el modelo
joblib.dump(model, "mosquitos_v2.sav")

['mosquitos_v2.sav']